In [1]:
import torch
from torch import nn

### CNN

In [2]:
# 构建一层卷积 - 模拟的是这张图：D:\workspaces\ai_study\day08\docs\卷积2.gif
conv1 = nn.Conv2d(in_channels=3, kernel_size=(3, 3), padding=1, stride=(2, 2), out_channels=2)
# 模拟一个5*5的input , torch的结构是[N, C, H, W]
X = torch.randn(1, 3, 5, 5)
# 查看output
print(f'Output的[N, C, H, W]：{conv1(X).shape}')

Output的[N, C, H, W]：torch.Size([1, 2, 3, 3])


In [3]:
# 通过conv1.weight.shape方式查看到的内容是[C_out, C_in, K_H, K_W]
C_out, C_in, K_H, K_W = conv1.weight.shape
# 参数量计算 params = (C_in * K_H * K_W + 1) * C_out
params_num = (C_in * K_H * K_W + 1) * C_out
print(f'参数量：{params_num}')

参数量：56


### BatchNorm

In [4]:
bn_layer = nn.BatchNorm2d(num_features=64)
print(bn_layer)  # eps:防除零, momentum:移动平均动量（用于推理时的统计）, affine:是否学习gamma和beta, track_running_states:是否跟踪全局均值/方差（推理时使用）

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


### 搭建一个LeNet5

In [5]:
import torch
from torch import nn

In [6]:
class LeNet5Cover(nn.Module):
    """
        自定义一个神经网络模型
    """
    def __init__(self, in_channels=1, output=10):
        """
            初始化函数
            
            1. Flatten参数的默认值说明
                nn.Flatten(start_dim = 1, end_dim = -1)默认值是1和-1. 意思是在[N, C, H, W]中，把从第1个维度开始到倒数第1个维度进行展平。
                也就是[N, C, H, W]展平成[N, C * H * W]
        """
        super().__init__()
        self.C1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.S2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.C3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.S4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fllatten = nn.Flatten()
        self.C5 = nn.Linear(in_features=400, out_features=120)
        self.F6 = nn.Linear(in_features=120, out_features=84)
        self.output = nn.Linear(in_features=84, out_features=output)
        
    def forward(self, x):
        """
            前向传播
        """
        x = self.C1(x)
        print(x.shape)
        x = self.S2(x)
        print(x.shape)
        x = self.C3(x)
        print(x.shape)
        x = self.S4(x)
        print(x.shape)
        x = self.fllatten(x)
        print(x.shape)
        x = self.C5(x)
        print(x.shape)
        x = self.F6(x)
        print(x.shape)
        x = self.output(x)
        print(x.shape)
        return x

In [7]:
# 实例化模型对象
lenet5 = LeNet5Cover(in_channels=1, output=10)

In [8]:
# 模拟一个Input的数据
X = torch.randn(1, 1, 32, 32)

In [9]:
y_pred = lenet5.forward(X)

torch.Size([1, 6, 28, 28])
torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 10, 10])
torch.Size([1, 16, 5, 5])
torch.Size([1, 400])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])


In [10]:
print(f'y_pred shape:{y_pred}\nshape is {y_pred.shape}')

y_pred shape:tensor([[-0.1017, -0.0560, -0.1336, -0.0871, -0.1800, -0.0230, -0.1144,  0.1432,
         -0.1151,  0.0446]], grad_fn=<AddmmBackward0>)
shape is torch.Size([1, 10])


In [11]:
y_pred.data

tensor([[-0.1017, -0.0560, -0.1336, -0.0871, -0.1800, -0.0230, -0.1144,  0.1432,
         -0.1151,  0.0446]])

In [12]:
y_pred.grad

C:\Users\63447\AppData\Local\Temp\ipykernel_13500\2470479642.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  y_pred.grad


In [13]:
lenet5

LeNet5Cover(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (S2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (S4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fllatten): Flatten(start_dim=1, end_dim=-1)
  (C5): Linear(in_features=400, out_features=120, bias=True)
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (output): Linear(in_features=84, out_features=10, bias=True)
)

In [18]:
class LeNet5Cover_Gai(nn.Module):
    """
        自定义一个神经网络模型 - 对上面的神经网络进行了改良.
        改良：做了流水线
    """
    def __init__(self, in_channels=1, output=10):
        """
            初始化函数
            
            1. Flatten参数的默认值说明
                nn.Flatten(start_dim = 1, end_dim = -1)默认值是1和-1. 意思是在[N, C, H, W]中，把从第1个维度开始到倒数第1个维度进行展平。
                也就是[N, C, H, W]展平成[N, C * H * W]
        """
        super().__init__()
        self.pipline = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Flatten(),
            nn.Linear(in_features=400, out_features=120),
            nn.Linear(in_features=120, out_features=84),
            nn.Linear(in_features=84, out_features=output)
        )

    def forward(self, x):
        """
            前向传播
        """
        return self.pipline(x)

In [19]:
lenet5_gai = LeNet5Cover_Gai()

In [20]:
X = torch.randn(1, 1, 32, 32)

In [21]:
y_pred = lenet5_gai.forward(X)
print(y_pred.shape)
print(lenet5_gai)

torch.Size([1, 10])
LeNet5Cover_Gai(
  (pipline): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=400, out_features=120, bias=True)
    (6): Linear(in_features=120, out_features=84, bias=True)
    (7): Linear(in_features=84, out_features=10, bias=True)
  )
)
